# __Imports__

In [158]:
import pandas as pd
import numpy as np
import os
import json

# __Data Import and Shape__

<a href="../data/raw/data.csv">data.csv</a> is a pre-processed dataset formed from the merge of <a href="../data/raw/data.csv">metrics.csv</a>, <a>survey.csv</a>, and <a href="../data/raw/data.csv">violence.csv</a>. It was pre-processed in Excel due to original databases not being in correct tabular format (multiple rows for column titles).

In [159]:
relative_path = os.path.join('..', 'data', 'raw', 'data.csv')

df = pd.read_csv(relative_path)
df.shape

C:\Users\Jose Avelar\AppData\Local\Temp\ipykernel_13740\2623336348.py:3: DtypeWarning: Columns (16,17,18,19,20,21,22,23,165) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(relative_path)


(6025, 174)

# __Column Types__

Assing the majority type to each column with mixed types.

In [160]:
for column in [16, 17, 18, 19, 20, 21, 22, 23, 165]:
    majority_type = df.iloc[:,column].apply(lambda x: type(x)).value_counts().idxmax()
    print(majority_type)

<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>


In [161]:
for column in [16, 17, 18, 19, 20, 21, 22, 23, 165]:
    df.iloc[:,column] = pd.to_numeric(df.iloc[:,column], errors='coerce')

# __Column Names__

In [162]:
column_names = df.columns.tolist()
column_dict = dict.fromkeys(column_names)
column_dict

{'A.02. Código del centro educativo según último acuerdo de acreditación:': None,
 'A.01. Nombre del centro educativo, según último acuerdo de acreditacion:': None,
 'A.03. Distrito educativo según última organización de la Dirección Departamental de Educación:': None,
 'A.04. Sector:': None,
 'A.04.a. Si marco sector privado, ¿Recibe Subsidio?:': None,
 'A.05. Organismo de administración:': None,
 'A.06. Zona:': None,
 'Código del Departamento': None,
 'A.07. Departamento:': None,
 'Código del Municipio': None,
 'A.08. Municipio:': None,
 'A.09. Nombre del cantón:': None,
 'A.10. Nombre del caserío:': None,
 'A.11. Dirección actual:': None,
 'A.12. ¿El centro educativo, se encuentra ubicado en una comunidad indígena?': None,
 'B.01. Proporcionó estos datos:': None,
 'Apoyo que recibe: Asistencia Técnica;No recibe apoyo': None,
 'Apoyo que recibe: Económico Monetario;No recibe apoyo': None,
 'Apoyo que recibe: Material Didáctico;No recibe apoyo': None,
 'Apoyo que recibe: Mobiliario y 

Extract column names to a JSON editor to manually assign names.

In [163]:
# Write column_dict to a JSON file with UTF-8 encoding
with open('../data/interim/column_dict.json', 'w', encoding='utf-8') as f:
    json.dump(column_dict, f, ensure_ascii=False)

At this point, I manually assigned appropiate English names to all columns. After assignment, open JSON to reassign column names.

In [164]:
# Open the JSON file and make list of values
with open('../data/interim/edit_column_dict.json', 'r', encoding='utf-8') as f:
    edit_column_dict = json.load(f)

# Assign new column names to df
df.columns = list(edit_column_dict.values())

# Print column names
list(df.columns)

['School ID',
 'School Name',
 'School District',
 'Sector',
 'If private, does it receive subsidy?',
 'Administrative Body',
 'Zone',
 'Department Code',
 'Department Name',
 'Municipality Code',
 'Municipality Name',
 'Canton Name',
 'Hamlet Name',
 'Address',
 'Does the school belong to an indigenous community?',
 'Who provided the data?',
 'Does not receive technical support.',
 'Does not receive economic support.',
 'Does not receive didactic material support.',
 'Does not receive furniture and equipment support.',
 'Does not receive infrastructure support.',
 'Does not receive teacher remuneration support.',
 'Does not receive construction material support.',
 'Does not receive food support.',
 'Funding: cafeterias.',
 'Funding: voluntary contributions.',
 'Funding: own activities.',
 'Funding: donations.',
 'Do you own your facilities?',
 'Water source: internal pipeline.',
 'Water source: river, lake, spring.',
 'Water source: rainwater.',
 'Water source: public sink.',
 'Water

# __Generating new features__

In [165]:
# Make a hard copy of the dataframe
new_df = df.copy()

## __Explanation__

Many of the current features can be combined linearly to form metrics that better represent the significance of a characteristics of each school.

Take for example the following features. These represent if the school has robotic kits, and if they are used by students.

In [166]:
list(df.iloc[:10, 108:136].columns)

['Lego robotics kit: good condition.',
 'Lego robotics kit: bad condition.',
 'Lego robotics kit: rented.',
 'Lego robotics kit: used by students.',
 'Rex robotics kit: good condition.',
 'Rex robotics kit: bad condition.',
 'Rex robotics kit: rented.',
 'Rex robotics kit: used by students.',
 'NXT robotics kit: good condition.',
 'NXT robotics kit: bad condition.',
 'NXT robotics kit: rented.',
 'NXT robotics kit: used by students.',
 'EV3 robotics kit: good condition.',
 'EV3 robotics kit: bad condition.',
 'EV3 robotics kit: rented.',
 'EV3 robotics kit: used by students.',
 'Chumchebot robotics kit: good condition.',
 'Chumchebot robotics kit: bad condition.',
 'Chumchebot robotics kit: rented.',
 'Chumchebot robotics kit: used by students.',
 'Make Block robotics kit: good condition.',
 'Make Block robotics kit: bad condition.',
 'Make Block robotics kit: rented.',
 'Make Block robotics kit: used by students.',
 'Other robotics kit: good condition.',
 'Other robotics kit: bad cond

From these features, we will create two features.

1. **Good condition robotic kit rate** measures the rate of good robotic kits out of all robotic kits owned by the school. A rate of 1 means that all robotic kits are in good condition, which is better.
2. **Robotic kit usage rate** measures how many robotic kits are used by students out of all robotic kits possesed by the school. A rate of 1 means that all robotic kits owned are being used by students, which is better.

$\text{Good condition robotic kit rate (GCRKR)} = \frac{\text{total robotic kits in good condition}}{\text{total robotic kits in good condition} + \text{total robotic kits in bad condition}} \in [0,1]$

$\text{Robotic kit usage rate (RKUR)} = \frac{\text{total robotic used by students}}{\text{total robotic kits in good condition} + \text{total robotic kits in bad condition} + \text{total rented robotic kits}} \in [0,1]$

## __New features__

\begin{equation}
\text{Good condition robotic kit rate (GCRKR)} = \frac{\text{total robotic kits in good condition}}{\text{total robotic kits in good condition} + \text{total robotic kits in bad condition}} \in [0,1]
\end{equation}

- **Def**: Measures the rate of good robotic kits out of all robotic kits owned by the school. A rate of 1 means that all robotic kits are in good condition, which is better.
- **Assumptions**: We assume the denominator represents all robotic kits owned by the school.

\begin{equation}
\text{Robotic kit usage rate (RKUR)} = \frac{\text{total robotic kits used by students}}{\text{total robotic kits in good condition} + \text{total robotic kits in bad condition} + \text{total rented robotic kits}} \in [0,1]
\end{equation}

- **Def**: Measures how many robotic kits are used by students out of all robotic kits possesed by the school. A rate of 1 means that all robotic kits owned are being used by students, which is better.
- **Assumptions**: We assume the denominator represents all robotic kits possesed by the school.

In [167]:
# Drop columns 108 to 135 from new_df
new_df.drop(new_df.iloc[:, 108:136], inplace=True, axis=1)

In [168]:
def compute_gcrkr_rkur(row):
    total_rk_good = 0
    total_rk_bad = 0
    total_rk_rented = 0
    total_rk_student = 0

    for j in range(108, 136, 4):
        if not np.isnan(row.iloc[j]):
            total_rk_good += row.iloc[j]
        if not np.isnan(row.iloc[j+1]):
            total_rk_bad += row.iloc[j+1]
        if not np.isnan(row.iloc[j+2]):
            total_rk_rented += row.iloc[j+2]
        if not np.isnan(row.iloc[j+3]):
            total_rk_student += row.iloc[j+3]
    
    good_bad = total_rk_good + total_rk_bad
    all_rk = (total_rk_good + total_rk_bad + total_rk_rented)

    gcrkr = 0 if good_bad == 0 else total_rk_good / good_bad
    rkur = 0 if all_rk == 0 else total_rk_student / all_rk
    
    return pd.Series([gcrkr, rkur])

new_df[['GCRKR', 'RKUR']] = df.apply(compute_gcrkr_rkur, axis=1)

In [169]:
new_df[['GCRKR', 'RKUR']].head()

,GCRKR,RKUR
0,0.0,0.0
1,0.0,0.0
2,1.0,1.0
3,0.0,0.0
4,0.0,0.0


\begin{equation}
\text{Switch "Does not receive..." to "Receive..." features}
\end{equation}

- **Def**: A simple switch between 1s and 0s.

In [175]:
list(df.iloc[:10, 16:24].columns)

['Does not receive technical support.',
 'Does not receive economic support.',
 'Does not receive didactic material support.',
 'Does not receive furniture and equipment support.',
 'Does not receive infrastructure support.',
 'Does not receive teacher remuneration support.',
 'Does not receive construction material support.',
 'Does not receive food support.']

In [171]:
new_df.drop(['Does not receive technical support.',
 'Does not receive economic support.',
 'Does not receive didactic material support.',
 'Does not receive furniture and equipment support.',
 'Does not receive infrastructure support.',
 'Does not receive teacher remuneration support.',
 'Does not receive construction material support.',
 'Does not receive food support.'], inplace=True, axis=1)

In [172]:
new_df['Receives technical support'] = df.iloc[:, 16].apply(lambda x: 0 if x == 1 else 1)
new_df['Receives economic support'] = df.iloc[:, 17].apply(lambda x: 0 if x == 1 else 1)
new_df['Receives didactic material support'] = df.iloc[:, 18].apply(lambda x: 0 if x == 1 else 1)
new_df['Receives furniture and equipment support'] = df.iloc[:, 19].apply(lambda x: 0 if x == 1 else 1)
new_df['Receives infrastructure support'] = df.iloc[:, 20].apply(lambda x: 0 if x == 1 else 1)
new_df['Receives teacher renumeration support'] = df.iloc[:, 21].apply(lambda x: 0 if x == 1 else 1)
new_df['Receives construction material support'] = df.iloc[:, 22].apply(lambda x: 0 if x == 1 else 1)
new_df['Receives food support'] = df.iloc[:, 23].apply(lambda x: 0 if x == 1 else 1)

\begin{equation}
\text{Total internal funding (TIF)} = \text{total funding from cafeterias} + \text{total funding from own activities} \in \mathbb{R}
\end{equation}

- **Def**: All funding coming from internal operations at the school.
- **Assumptions**: We assume "total funding from own activities" represents total funding obtained from school's operations.

\begin{equation}
\text{Total external funding (TEF)} = \text{total funding from voluntary contributions} + \text{total funding from donations} \in \mathbb{R}
\end{equation}

- **Def**: All funding coming from external operations at the school.
- **Assumptions**: We assume its two parameters constitute all external funding to the school.

In [178]:
list(df.iloc[:10, 24:28].columns)

['Funding: cafeterias.',
 'Funding: voluntary contributions.',
 'Funding: own activities.',
 'Funding: donations.']

In [ ]:
new_df.drop(['Funding: cafeterias', 'Funding: voluntary contributions.', 'Funding: own activities.', 'Funding: donations.'], inplace=True, axis=1)

In [183]:
# Remove commas from columns 24 to 27
df.iloc[:, 24:28] = df.iloc[:, 24:28].replace(',', '', regex=True)

# Make columns 24 to 27 into floats
df.iloc[:, 24:28] = df.iloc[:, 24:28].astype(float)

In [186]:
def  compute_funding(row):
    tif = 0
    tef = 0
    
    if not np.isnan(row.iloc[24]) and not np.isnan(row.iloc[26]):
        tif = row.iloc[24] + row.iloc[26]
    if not np.isnan(row.iloc[25]) and not np.isnan(row.iloc[27]):
        tef = row.iloc[25] + row.iloc[27]
    
    return pd.Series([tif, tef])

new_df[['TIF', 'TEF']] = df.apply(compute_funding, axis=1)

In [187]:
new_df[['TIF', 'TEF']].head()

,TIF,TEF
0,4573.89,727338.03
1,2652.00,0.00
2,3919.64,2450.50
3,2000.00,300.00
4,245.00,0.00


\begin{equation}
\text{Water source (WS)} = 
\begin{cases} 
    \text{Internal Pipeline} \\
    \text{River, Lake or Spring} \\
    \text{Rainwater} \\
    \text{Public Sink} \\
    \text{Well} \\
    \text{Pipe}
\end{cases}
\end{equation}

In [191]:
list(df.iloc[:10, 28:36].columns)

['Do you own your facilities?',
 'Water source: internal pipeline.',
 'Water source: river, lake, spring.',
 'Water source: rainwater.',
 'Water source: public sink.',
 'Water source: well.',
 'Water source: pipe.',
 'Does the school have electrical installations?']